# Impact of Bedmachine on ice flow modelling thourgh version 1-5
### Group Project International Summer School in Glaciology, McCarthy, June 7-17 2024

## **Is Bedmachine getting better?**

NASA’s Operation IceBridge (2009-2019) changed our understanding of the subglacial topography beneath the Greenland Ice Sheet. Radar-derived Ice thickness along flightlines have been gridded using kriging and algorithms based on mass conservation, resulting in the now widely-used ”BedMachine Greenland”. A manuscript in 2016 showed that BedMachine Greenland significantly improves the ability of ice flow models to reproduce the flow patterns of Greenland’s outlet glaciers. The study was done using BedMachine version 1, however, the release version is version 5. In this project we will investigate the changes in ice thickness in BedMachine throughout the different versions, and how it affects an ice flow model’s fidelity in reproducing outlet glacier flow. 



In [ ]:
import pylab as plt
import xarray as xr
import geopandas as gp
from functools import partial

In [ ]:
from glacier_flow_tools.profiles import process_profile
from glacier_flow_tools.utils import preprocess_nc

In [ ]:
profile_resolution = 1_000  # m

## Load observed and simulated velocities

In [ ]:
itslive_ds = xr.open_dataset("data/its_live_jak.nc")
exp_ds = xr.open_mfdataset("../tests/data/velsurf_mag_gris_g9000m_id_*_0_50.nc",
                           preprocess=partial(preprocess_nc,
                           drop_dims=["z", "zb", "nv4"],
                           drop_vars=["timestamp", "shelfbtemp", "effective_ice_surface_temp", "ice_surface_temp", "hardav", "nv4"]), 
                           concat_dim="exp_id",
                           combine="nested",
                           chunks="auto",
                           engine="h5netcdf",
                           decode_times=False,
                           parallel=True
                          )

## Load profile shapefile

In [ ]:
profiles_gp = gp.read_file("../glacier_flow_tools/data/greenland-flux-gates-5.gpkg").rename(columns={"id": "profile_id", "name": "profile_name"})
geom = profiles_gp.segmentize(profile_resolution)
profiles_gp = gp.GeoDataFrame(profiles_gp, geometry=geom)
profiles_gp = profiles_gp[["profile_id", "profile_name", "geometry"]]

profile_gp = [p for _, p in profiles_gp.iterrows()][1]

## Plot map-plane view of Jakobshavn gate

In [ ]:
fig, ax = plt.subplots()
itslive_ds["v"].plot(cmap="speed_colorblind", vmin=0, vmax=1500, ax=ax, cbar_kwargs={"shrink": 0.75})
profiles_gp.plot(ax=ax, color="k")
ax.set_title("Jakobshavn Flux Gate\nObserved Speed")

In [ ]:
jak_profile = process_profile(profile_gp,
                itslive_ds,
                exp_ds,
                compute_profile_normal=True,
                stats_kwargs={"obs_var": "v", "sim_var": "velsurf_mag"})

## Plot profile

In [ ]:
jak_profile.profiles.plot(obs_error_var=None, interactive=True, figsize=(8, 8), fontsize=12, 
                          obs_kwargs={"lw": 2, "color": "k", "marker": "o", "ms": 5}, sim_kwargs={"lw": 2, "marker": "o", "ms": 5})

## Tasks

- Calculate mean absolute error between observations and all simulations